In [2]:
## Stanford heart transplant censored data study

## TRUNCATE THE DATA SO THAT THERE ARE MORE EFFECTS DOWNSTREAM WITH THE POSTERIOR DISTRIBUTIONS

import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

np.random.seed(40) # For reproducibility

stanfordHeartData = pd.read_csv('heart_transplant.csv')

# Regressing log survival time on age to get sample variance of errors

aliveIndices = stanfordHeartData.index[stanfordHeartData['survived'] == 'alive'].tolist()
deadIndices = stanfordHeartData.index[stanfordHeartData['survived'] == 'dead'].tolist()
# Truncate the data 
truncatedAlive = 28
truncatedDead = 20
aliveIndices = random.sample(aliveIndices, truncatedAlive)
deadIndices = random.sample(deadIndices, truncatedDead)

deadSurvivalTimes = stanfordHeartData['survtime'][deadIndices]
logDeadSurvivalTimes = np.array(np.log(deadSurvivalTimes))
deadAge = np.array(stanfordHeartData['age'][deadIndices])

aliveCensoredEventTimes = stanfordHeartData['survtime'][aliveIndices]
logAliveCensoredEventTimes = np.array(np.log(aliveCensoredEventTimes))
aliveAge = np.array(stanfordHeartData['age'][aliveIndices])
numCensoredEventTimes = len(aliveCensoredEventTimes)
allAge = np.append(deadAge.reshape(-1, 1), aliveAge.reshape(-1, 1), 0)

model = LinearRegression()

# Reshape the arrays

deadAgeReshaped = deadAge.reshape(-1, 1)
logDeadSurvivalTimesReshaped = logDeadSurvivalTimes.reshape(-1, 1)

model.fit(deadAgeReshaped, logDeadSurvivalTimesReshaped)
beta0 = model.intercept_[0]
beta1 = model.coef_[0][0]
yHat = model.predict(deadAgeReshaped)
residuals = yHat - logDeadSurvivalTimesReshaped # Flatten the predicted values
nonAugmentedResidualVariance = np.var(residuals)

In [ ]:
# Draw o^2* from the chi square distribution

numObservedData = len(deadSurvivalTimes)
numIterations = 50
numImputations = 5000
currentResidualVariance = (numObservedData - 1) * nonAugmentedResidualVariance / np.random.chisquare(numObservedData - 1, numImputations)
designMatrix = np.ones([numObservedData, 2])
designMatrix[:, 1] = deadAgeReshaped.flatten()
jointPosteriorApprox = np.zeros([numImputations, 3])
jointPosteriorApprox[:, 2] = currentResidualVariance.reshape(-1, 1).flatten()
numTotalDatapoints = len(stanfordHeartData['age'])

# Initialize the approximation to the posterior distribution by sampling the error variance
# from the linear regression model that does not include the right censored data

for iiImputation in range(numImputations):

    varStar = currentResidualVariance[iiImputation]
    covarianceMatrix = np.linalg.inv(varStar * designMatrix.T @ designMatrix) 
    tempBetaStars = np.random.multivariate_normal([beta0, beta1], covarianceMatrix, 1).reshape(1, -1)
    jointPosteriorApprox[iiImputation, 0:2] = tempBetaStars


## Now start the actual algorithm

augmentedSurvivalTimes = np.zeros([numCensoredEventTimes, numImputations]) ## row = data point, column = imputation number
quantiles = np.array([.025,.50, .975])
convergenceBeta0 = np.zeros([len(quantiles), numIterations])
convergenceBeta1 = np.zeros([len(quantiles), numIterations])
convergenceSigmaSq = np.zeros([len(quantiles), numIterations])

for iiIteration in range(numIterations):

    if iiIteration % 10 == 0:

        print('Iteration: ', iiIteration)

    for jjImputation in range(numImputations):

        # Randomly select from the mixture of posterior distributions

        randomIndex = random.randint(0, numImputations - 1)
        thetaStar = jointPosteriorApprox[randomIndex, :]
        sigmaSqStar = thetaStar[-1]
        beta0Star = thetaStar[0]
        beta1Star = thetaStar[1]

        # Generate the augmented data by sampling errors and adding to right censored event time

        for kkCensoredEventTime in range(numCensoredEventTimes):

            predictedSurvivalTime = (beta0Star + beta1Star * aliveAge[kkCensoredEventTime])
            predictedError = predictedSurvivalTime - logAliveCensoredEventTimes[kkCensoredEventTime]
            drawFromConditionalErroDistribution = np.random.normal(0, sigmaSqStar)

            while drawFromConditionalErroDistribution < predictedError:

                drawFromConditionalErroDistribution = np.random.normal(0, sigmaSqStar)

            augmentedSurvivalTimes[kkCensoredEventTime, jjImputation] = predictedSurvivalTime + drawFromConditionalErroDistribution

        # Update the approximation to the posterior 
        # Randomly select the augmented data to use, then do linear regression, then sample the sigmaSqStar, then beta0 and beta1

    for jjImputation in range(numImputations):

        randomIndex = random.randint(0, numImputations - 1)
        tempAugmentedData = augmentedSurvivalTimes[:, randomIndex]
        tempAugmentedDesignMatrix = np.append(np.ones([numCensoredEventTimes, 1]), aliveAge.reshape(-1, 1), 1)
        tempTotalDesignMatrix = np.append(designMatrix, tempAugmentedDesignMatrix, 0)
        tempAugmentedSurvivalTimes = np.append(logDeadSurvivalTimes.reshape(-1, 1), tempAugmentedData.reshape(-1, 1), 0)

        model.fit(tempTotalDesignMatrix[:, -1].reshape(-1, 1), tempAugmentedSurvivalTimes)

        ## Need to sample the coefficients from the posteriors for beta0 beta1 sigmaStar from augmented linear model and add to array
        # The posterior for the betas is normal centered around the least squares estimates with covar matrix inv(sample o^2 * X_T @ X)
        
        beta0 = model.intercept_[0]
        beta1 = model.coef_[0][0]
        yHat = model.predict(tempTotalDesignMatrix[:, -1].reshape(-1, 1))
        residuals = yHat - tempAugmentedSurvivalTimes
        sampleResidualVariance = np.var(residuals)
        sigmaSquaredStar = (numTotalDatapoints - 1) * sampleResidualVariance / np.random.chisquare(numTotalDatapoints - 1, 1)

        tempCovarianceMatrix = np.linalg.inv(sigmaSquaredStar * tempTotalDesignMatrix.T @ tempTotalDesignMatrix) 
        betasStar = np.random.multivariate_normal([beta0, beta1], tempCovarianceMatrix, 1).reshape(1, -1)
        jointPosteriorApprox[iiImputation, -1] = sigmaSquaredStar
        jointPosteriorApprox[iiImputation, 0:2] = betasStar

    convergenceBeta0[:, iiIteration] = np.quantile(jointPosteriorApprox[:, 0], quantiles, axis = 0)
    convergenceBeta1[:, iiIteration] = np.quantile(jointPosteriorApprox[:, 1], quantiles, axis = 0)
    convergenceSigmaSq[:, iiIteration] = np.quantile(jointPosteriorApprox[:, -1], quantiles, axis = 0)

## MAKE LAST SAMPLE FROM POSTERIOR DISTRIBUTION LARGER THAN THE REST, I.E. 10,000 SAMPLES FROM LAST POSTERIOR DISTRIBUTION
            
## Each iteration gives you an approximation to the posterior distribution
## Impute the m = 500 values for augmented data, then with the augmented data, sample from the mixture
## of posterior distributions (randomly sample augmented data, use that to calculate and sample from that beta posterior)
# So the jointPosteriorApprox matris is exactly that, a sample from the last posterior distribution 


In [ ]:
# Plot the joint distribution of beta0 and beta1 together

beta0Distribution = jointPosteriorApprox[:, 0]
beta1Distribution = jointPosteriorApprox[:, 1]

print('Covariance for augmented joint posterior: ', np.cov(beta0Distribution, beta1Distribution))

plt.scatter(beta1Distribution, beta0Distribution, color = 'black')
plt.xlabel('$\\beta_1$')
plt.ylabel('$\\beta_0$')
plt.xlim(-.05, .15)
plt.ylim(-2, 6)
plt.title('Augmented Posterior Joint Distribution Approximation for $\\beta_0$ and $\\beta_1$')
plt.show()

model.fit(deadAgeReshaped, logDeadSurvivalTimesReshaped)
beta0 = model.intercept_[0]
beta1 = model.coef_[0][0]
yHat = model.predict(deadAgeReshaped)
residuals = yHat - logDeadSurvivalTimesReshaped # Flatten the predicted values
nonAugmentedResidualVariance = np.var(residuals)
numSamples = 1000
nonAugmentedPosteriorApprox = np.zeros([numSamples, 3])


for iiSample in range(numSamples):

    nonAugmentedSampleResidualVariance = (numObservedData - 1) * nonAugmentedResidualVariance / np.random.chisquare(numObservedData - 1, 1)
    nonAugCovarMatrix = np.linalg.inv(nonAugmentedSampleResidualVariance * designMatrix.T @ designMatrix)
    betasStar = np.random.multivariate_normal([beta0, beta1], covarianceMatrix, 1).reshape(1, -1)
    nonAugmentedPosteriorApprox[iiSample, 0:2] = betasStar
    nonAugmentedPosteriorApprox[iiSample, -1] = nonAugmentedSampleResidualVariance

beta0Distribution = nonAugmentedPosteriorApprox[:, 0]
beta1Distribution = nonAugmentedPosteriorApprox[:, 1]
plt.scatter(beta1Distribution, beta0Distribution, color = 'black')
plt.xlabel('$\\beta_1$')
plt.ylabel('$\\beta_0$')
plt.xlim(-.05, .15)
plt.ylim(-2, 6)
plt.title('Nonaugmented Posterior Joint Distribution Approximation for $\\beta_0$ and $\\beta_1$')
plt.show()

## PRINT THE SAMPLE COVARIANCES FOR THE CORRESPONDING PLOTS IN THE 

print('Covariance for nonaugmented joint posterior: ', np.cov(beta0Distribution, beta1Distribution))
# fprintf('Covariance for nonaugmented joint posterior: )

In [ ]:
# Plot the mean regression lines for augmented and nonaugmented posterior distributions

x = np.linspace(np.min(deadAge), np.max(deadAge), 1000)
meanAugmentedB0 = np.mean(jointPosteriorApprox[:, 0])
meanAugmentedB1 = np.mean(jointPosteriorApprox[:, 1])
plt.plot(x, meanAugmentedB0 + meanAugmentedB1 * x, color = 'r', label = 'Augmented')
plt.xlabel('Age')
plt.ylabel('Log Survival Time')
plt.title('Mean Regression Lines for Nonaugmented and Augmented Analysis')

meanNonaugmentedB0 = np.mean(nonAugmentedPosteriorApprox[:, 0])
meanNonaugmentedB1 = np.mean(nonAugmentedPosteriorApprox[:, 1])
plt.plot(x, meanNonaugmentedB0 + meanNonaugmentedB1 * x, color = 'blue', label = 'Nonaugmented')
plt.legend()
plt.show()

# Plot the exponential transformation

plt.plot(x, np.exp(meanAugmentedB0 + meanAugmentedB1 * x), color = 'r', label = 'Augmented')
plt.plot(x, np.exp(meanNonaugmentedB0 + meanNonaugmentedB1 * x), color = 'blue', label = 'Nonaugmented')
plt.xlabel('Age')
plt.ylabel('Survival Time')
plt.legend()
plt.title('Exponential Transformation for Nonaugmented and Augmented Regression Lines')
plt.show()

In [ ]:
augmentedBeta1Posterior = jointPosteriorApprox[:, 1]
augmentedBeta0Posterior = jointPosteriorApprox[:, 0]
augmentedSigmaSqPosterior = jointPosteriorApprox[:, -1]

# Sample the posterior for the nonaugmented data

plt.hist(augmentedBeta1Posterior, bins = 30, density = True, color = 'skyblue', edgecolor = 'black', alpha = .5, label = 'augmented')
nonAugmentedBeta1Posterior = nonAugmentedPosteriorApprox[:, 1]
plt.hist(nonAugmentedBeta1Posterior, bins = 30, density = True, color = 'red', edgecolor = 'black', alpha = .5, label = 'nonaugmented')
plt.rc('text', usetex = True)
plt.xlabel('$\\beta_1$')
plt.ylabel('Density')
plt.title('Comparing Posterior Distributions for Slope of Linear Regression Model')
plt.legend()
plt.show()

plt.hist(augmentedBeta0Posterior, bins = 30, density = True, color = 'skyblue', edgecolor = 'black', alpha = .5, label = 'augmented')
nonAugmentedBeta0Posterior = nonAugmentedPosteriorApprox[:, 0]
plt.hist(nonAugmentedBeta0Posterior, bins = 30, density = True, color = 'red', edgecolor = 'black', alpha = .5, label = 'nonaugmented')
plt.rc('text', usetex = True)
plt.xlabel('$\\beta_0$')
plt.ylabel('Density')
plt.title('Comparing Posterior Distributions for Intercept of Linear Regression Model')
plt.legend()
plt.show()

plt.hist(augmentedSigmaSqPosterior, bins = 30, density = True, color = 'skyblue', edgecolor = 'black', alpha = .5, label = 'augmented')
nonAugmentedSigmaSqPosterior = nonAugmentedPosteriorApprox[:, -1]
plt.hist(nonAugmentedSigmaSqPosterior, bins = 30, density = True, color = 'red', edgecolor = 'black', alpha = .5, label = 'nonaugmented')
plt.rc('text', usetex = True)
plt.xlabel('$\\sigma^2$')
plt.ylabel('Density')
plt.title('Comparing Posterior Distributions for Error Variance of Linear Regression Model')
plt.legend()
plt.show()

## overlay the posterior distribution without the augmented data with the posterior distribution with the augmented data

## plot the mean regression line for the augmented posterior (mean of b0 and b1) next to the mean regression line for the non augmented posterior
# which is just the least squares estimate that you got before the algorithm, compare the differences 

# Can look at more examples for data augmentation, or can look at the math behind convergence, or can DO CARD SHUFFLING ANALYSIS!